In [ ]:
#| default_exp rolling

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from math import sqrt
from typing import Callable, Optional, Tuple

import numpy as np
from numba import njit  # type: ignore

from window_ops.utils import _gt, _lt, _validate_rolling_sizes, first_not_na

In [ ]:
from typing import Union

import pandas as pd

def prepend_nans(
        collection: Union[np.ndarray, pd.Series],
        number_of_nans: int) -> Union[np.ndarray, pd.Series]:
    nans = np.full(number_of_nans, np.nan, dtype=np.float64)
    if isinstance(collection, np.ndarray):
        return np.hstack((nans, collection))
    if isinstance(collection, pd.Series):
        return pd.concat([pd.Series(nans), collection])
    raise ValueError(f'Collection must be np.ndarray or pd.Series, got: {type(collection)}')

In [ ]:
import random

import pandas as pd

np.random.seed(0)
number_of_nans = 10

array = np.random.rand(100)
array_with_nans = prepend_nans(array, number_of_nans)
series = pd.Series(array)
series_with_nans = prepend_nans(series, number_of_nans)
all_nans_array = np.full(100, np.nan)

## Regular

In [ ]:
#| exporti
def rolling_docstring(*args, **kwargs) -> Callable:
    base_docstring = """
        Compute the {} over the last non-na window_size samples of the
        input array starting at min_samples.
    """
    def docstring_decorator(function: Callable):
        function.__doc__ = base_docstring.format(function.__name__)
        return function
        
    return docstring_decorator(*args, **kwargs)

In [ ]:
def test_rolling_vs_pandas(rolling: Callable,
                           pandas_aggregation: str,
                           lower_bound_for_min_samples: int = 1,
                           non_na_values: int = 5) -> None:
    
    window_size = random.randint(3, 10)
    min_samples = random.randint(2, window_size)
    
    # expanding for [min_samples, window_size), rolling for [window_size, n_samples]
    np.testing.assert_allclose(
        rolling(array, window_size, min_samples=lower_bound_for_min_samples), 
        series.rolling(window_size, min_periods=lower_bound_for_min_samples).agg(pandas_aggregation)
    )

    # arbitrary min_samples and window_size
    np.testing.assert_allclose(
        rolling(array, window_size, min_samples=min_samples), 
        series.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
    )

    # min_samples = window_size
    np.testing.assert_allclose(
        rolling(array, window_size),
        series.rolling(window_size).agg(pandas_aggregation)
    )

    # skip nas
    np.testing.assert_allclose(
        rolling(array_with_nans, window_size, min_samples=min_samples),
        series_with_nans.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
    )

    # |non-na-values| < min_samples
    np.testing.assert_allclose(
        rolling(
            array_with_nans[:number_of_nans + non_na_values],
            window_size=non_na_values+2,
            min_samples=non_na_values+1),
        all_nans_array[:number_of_nans + non_na_values]
    )

    # min_samples < |non-na-values| < window_size
    np.testing.assert_allclose(
        rolling(
            array_with_nans[:number_of_nans + lower_bound_for_min_samples+2], 
            window_size=lower_bound_for_min_samples+1,
            min_samples=lower_bound_for_min_samples),
        np.hstack((
            all_nans_array[:number_of_nans], 
            rolling(
                array_with_nans[number_of_nans : number_of_nans+lower_bound_for_min_samples+2], 
                window_size=lower_bound_for_min_samples+1, 
                min_samples=lower_bound_for_min_samples)
        ))
    )

    # all-nan -> all-nan
    np.testing.assert_equal(
        rolling(all_nans_array, window_size, min_samples=min_samples),
        all_nans_array
    )    

In [ ]:
#| export
@njit
@rolling_docstring
def rolling_mean(input_array: np.ndarray,
                 window_size: int,
                 min_samples: Optional[int] = None) -> np.ndarray:
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    
    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array
    
    accum = 0.
    upper_limit = min(start_idx + window_size, n_samples)
    for i in range(start_idx, upper_limit):
        accum += input_array[i]
        if i + 1 >= start_idx + min_samples:
            output_array[i] = accum / (i - start_idx + 1)
            
    for i in range(start_idx + window_size, n_samples):
        accum += input_array[i] - input_array[i - window_size]
        output_array[i] = accum / window_size

    return output_array

In [ ]:
test_rolling_vs_pandas(rolling=rolling_mean, pandas_aggregation='mean')

In [ ]:
#| exporti
@njit
def _rolling_std(input_array: np.ndarray, 
                 window_size: int,
                 min_samples: Optional[int] = None) -> Tuple[np.ndarray, float, float]:
    """Computes the rolling standard deviation using Welford's online algorithm.
    
    Reference: https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_online_algorithm"""
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    if min_samples < 2:  # type: ignore
        raise ValueError('min_samples must be greater than 1.')

    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array, 0, 0

    prev_avg = 0.
    curr_avg = input_array[start_idx]
    m2 = 0.
    upper_limit = min(start_idx + window_size, n_samples)
    for i in range(start_idx + 1, upper_limit):
        prev_avg = curr_avg
        curr_avg = prev_avg + (input_array[i] - prev_avg) / (i - start_idx + 1)
        m2 += (input_array[i] - prev_avg) * (input_array[i] - curr_avg)
        if i + 1 >= start_idx + min_samples:
            output_array[i] = sqrt(m2 / (i - start_idx))
            
    for i in range(start_idx + window_size, n_samples):
        prev_avg = curr_avg
        new_minus_old = input_array[i] - input_array[i-window_size]
        curr_avg = prev_avg + new_minus_old / window_size
        m2 += new_minus_old * (input_array[i] - curr_avg + input_array[i-window_size] - prev_avg)
        output_array[i] = sqrt(m2 / (window_size - 1))
        
    return output_array, curr_avg, m2

In [ ]:
#| export
@njit
@rolling_docstring
def rolling_std(input_array: np.ndarray, 
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    output_array, _, _ = _rolling_std(input_array, window_size, min_samples)
    return output_array

In [ ]:
test_rolling_vs_pandas(rolling=rolling_std, pandas_aggregation='std', lower_bound_for_min_samples=2)

In [ ]:
#| exporti
@njit 
def _rolling_comp(comp: Callable,
                  input_array: np.ndarray, 
                  window_size: int,
                  min_samples: Optional[int] = None) -> np.ndarray:
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    
    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array
    
    upper_limit = min(start_idx + window_size, n_samples)
    pivot = input_array[start_idx]
    for i in range(start_idx, upper_limit):
        if comp(input_array[i], pivot) > 0:
            pivot = input_array[i]
        if i + 1 >= start_idx + min_samples:
            output_array[i] = pivot
    
    for i in range(start_idx + window_size, n_samples):
        pivot = input_array[i]
        for j in range(1, window_size):
            if comp(input_array[i - j], pivot) > 0:
                pivot = input_array[i - j]
        output_array[i] = pivot
    return output_array

In [ ]:
#| export
@njit
@rolling_docstring
def rolling_max(input_array: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    return _rolling_comp(_gt, input_array, window_size, min_samples)

In [ ]:
test_rolling_vs_pandas(rolling_max, 'max')

In [ ]:
#| export
@njit
@rolling_docstring
def rolling_min(x: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    return _rolling_comp(_lt, x, window_size, min_samples)

In [ ]:
test_rolling_vs_pandas(rolling_min, 'min')

## Seasonal

In [ ]:
#| exporti
@njit
def _seasonal_rolling_op(rolling_op: Callable,
                         input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray: 
    n_samples = input_array.size
    output_array = np.full_like(input_array, np.nan)
    for season in range(season_length):
        output_array[season::season_length] = rolling_op(input_array[season::season_length], window_size, min_samples)
    return output_array

In [ ]:
from functools import partial

season_length = 7


def test_seasonal_rolling_vs_pandas(seasonal_rolling: Callable,
                                    pandas_aggregation: str,
                                    lower_bound_for_min_samples: int = 1,
                                    non_na_values: int = 5) -> None:
    
    window_size = random.randint(3, 4)
    min_samples = random.randint(2, window_size)
    
    # expanding for [min_samples, window_size), rolling for [window_size, n_samples]
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size, min_samples=lower_bound_for_min_samples), 
        grouped_series.transform(
            lambda x: x.rolling(window_size, min_periods=lower_bound_for_min_samples).agg(pandas_aggregation)
        )
    )

    # arbitrary min_samples and window_size
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size, min_samples=min_samples), 
        grouped_series.transform(
            lambda x: x.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
        )
    )

    # min_samples = window_size
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size),
        grouped_series.transform(
            lambda x: x.rolling(window_size).agg(pandas_aggregation))
    )

    # skip nas
    np.testing.assert_allclose(
        seasonal_rolling(array_with_nans, window_size=window_size, min_samples=min_samples),
        grouped_series_with_nans.transform(
            lambda x: x.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
        )
    )

    # |non-na-values| < min_samples
    np.testing.assert_allclose(
        seasonal_rolling(
            array_with_nans[:number_of_nans + non_na_values],
            window_size=non_na_values+2,
            min_samples=non_na_values+1),
        all_nans_array[:number_of_nans + non_na_values]
    )

    # min_samples < |non-na-values| < window_size
    np.testing.assert_allclose(
        seasonal_rolling(
            array_with_nans[:number_of_nans + lower_bound_for_min_samples + 2*season_length], 
            window_size=lower_bound_for_min_samples + season_length,
            min_samples=lower_bound_for_min_samples),
        np.hstack((
            all_nans_array[:number_of_nans], 
            seasonal_rolling(
                array_with_nans[number_of_nans : number_of_nans + lower_bound_for_min_samples + 2*season_length], 
                window_size=lower_bound_for_min_samples + season_length,
                min_samples=lower_bound_for_min_samples)
        ))
    )

    # all-nan -> all-nan
    np.testing.assert_equal(
        seasonal_rolling(all_nans_array, window_size=window_size, min_samples=min_samples),
        all_nans_array
    )

    
def get_seasons(season_length, n_samples):
    return np.arange(n_samples) % season_length

grouped_series = series.groupby(get_seasons(season_length, series.size))
grouped_series_with_nans = series_with_nans.groupby(get_seasons(season_length, series_with_nans.size))

In [ ]:
#| export
@njit
@rolling_docstring
def seasonal_rolling_mean(input_array: np.ndarray,
                          season_length: int,
                          window_size: int,
                          min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_mean, input_array, season_length, window_size, min_samples)

In [ ]:
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_mean, season_length=season_length), 'mean')

In [ ]:
#| export
@njit
@rolling_docstring
def seasonal_rolling_std(input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_std, input_array, season_length, window_size, min_samples)

In [ ]:
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_std, season_length=season_length), 'std', lower_bound_for_min_samples=2)

In [ ]:
#| export
@njit
@rolling_docstring
def seasonal_rolling_max(input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_max, input_array, season_length, window_size, min_samples)

In [ ]:
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_max, season_length=season_length), 'max')

In [ ]:
#| export
@njit
@rolling_docstring
def seasonal_rolling_min(x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_min, x, season_length, window_size, min_samples)

In [ ]:
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_min, season_length=season_length), 'min')